In [1]:
#import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [2]:
#torch.cuda.is_available()

In [3]:
model_path = r"D:/Python/AI_POC/New_python/defog"
model_name = "defog/sqlcoder"
off_load_path= r"D:/Python/AI_POC/New_python/defog/offload"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [5]:
#model = AutoModelForCausalLM.from_pretrained(    model_path)

In [6]:
import accelerate

In [7]:

#tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    # torch_dtype=torch.bfloat16,
    # load_in_8bit=True,
    load_in_4bit=False,
    #device_map="auto",
    use_cache=True,
    offload_folder= off_load_path
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
question = "Total Entry transactions for date 12th July 2023"

In [44]:
prompt = """### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the questioq `{question}`.
This query will run on a database whose schema is represented in this string:
CREATE TABLE ENEX_MASTER  --table for entry exit 
(
  EMA_SEQ                  NUMBER(5), -- Unique ID for each SPM_PERSON_NO
  SPM_PERSON_NO            NUMBER(12), --Unique number per each person
  NAT_CODE                 NUMBER(3),
  EBO_CODE                 NUMBER(4),
  ETT_CODE                 NUMBER(3),
  SFU_CODE                 NUMBER(2)          ,
  SDE_CODE                 NUMBER(3),
  EMA_TYPE                 NUMBER(1), ema_type 1 is entry, 2 is exit 
  EMA_YEAR                 NUMBER(4),
  EMA_DATE                 DATE, --entry or exit date based on ema_type
  EMA_CREATED_BY           VARCHAR2(10 BYTE), --if ema_created is E$GATE THEN Egate transaction else Eborder
  EMA_DEL_FLAG             NUMBER(1)          ,
  EMA_FLIGHT_TRIP_NO       VARCHAR2(10 BYTE),
  EMA_AIR_SHIP_CODE        NUMBER(10),
  AIRPORT_CODE             VARCHAR2(3 BYTE)
  );

CREATE TABLE NATIONALITIES  --table for nationalities lookup
(
  CODE                NUMBER(3)        ,
  COUNTRY             VARCHAR2(50 BYTE),
  COUNTRY_ENG         VARCHAR2(50 BYTE),
  SHORT_DESC          VARCHAR2(3 BYTE)
  );

CREATE TABLE PERSON_MASTER   --table for person details
(
  SPM_PERSON_NO            NUMBER(12)          ,--foreign key to column SPM_PERSON_NO  in enex_master
  NAT_CODE_CURR_NAT        NUMBER(3)           , --This is Nationality of person
  SPM_FULL_ANAME           VARCHAR2(80 BYTE), --full name of passenger in arabic
  SPM_FULL_ENAME           VARCHAR2(80 BYTE),--full name of passenger in english
  SPM_DOB                  DATE,  --date of birth of passenger
  SPM_GENDER               NUMBER(1), --gender of passenger 
  SPM_NATIONAL_ID          VARCHAR2(20 BYTE)
;

create table ENEX_BORDER  --table for border types
(
ebo_code number ,
ebo_type number , ---ebo_type =1 is Land Border,ebo_type=2 is Sea port,ebo_type= 3 it is Airport 
ebo_desc varchar2(80 byte)
);

-- ENEX_MASTER.SPM_PERSON_NO can be joined with PERSON_MASTER.SPM_PERSON_NO
-- PERSON_MASTER.NAT_CODE_CURR_NAT can be joined with NATIONALITIES.code
-- ENEX_MASTER.ebo_code can be joined with ENEX_border.ebo_code
--ebo_type =1 is Land Border
--ebo_type=2 is Sea port
--ebo_type= 3 it is Airport 
--SPM_GENDER =0 it is female
--SPM_GENDER = 1 it is male

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)

In [45]:
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

In [11]:
#inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

#inputs = tokenizer(prompt, return_tensors="pt")
#generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=5
)

IndentationError: unexpected indent (2811503374.py, line 5)

In [46]:
#inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

inputs = tokenizer(prompt, return_tensors="pt")
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    #max_new_tokens=400,
    do_sample=False,
    num_beams=5,
    max_length=1000
)

In [ ]:
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [14]:
torch.cuda.empty_cache()
# empty cache so that you do generate more results w/o memory crashing
# particularly important on Colab – memory management is much more straightforward
# when running on an inference service

NameError: name 'torch' is not defined

In [15]:
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT sum(enex_master.ema_seq) AS total_entry_transactions
FROM   enex_master
WHERE  enex_master.ema_date = '2023-07-12'
   and enex_master.ema_type = 1;


In [ ]:
print(outputs)

In [24]:
question = "Total Entry transactions OF Seaport for date 12th July 2023"

In [31]:
#inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

inputs = tokenizer(prompt, return_tensors="pt")
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    #max_new_tokens=400,
    do_sample=False,
    num_beams=5,
    max_length=1200
)

In [32]:
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [33]:
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT sum(enex_master.ema_seq) AS total_entry_transactions
FROM   enex_master join enex_border on enex_master.ebo_code = enex_border.ebo_code
WHERE  enex_master.ema_date = '2023-07-12'
   and enex_border.ebo_type = 2;


In [34]:
print(outputs)

["### Instructions:\nYour task is convert a question into a SQL query, given a Postgres database schema.\nAdhere to these rules:\n- **Deliberately go through the question and database schema word by word** to appropriately answer the question\n- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.\n- When creating a ratio, always cast the numerator as float\n\n### Input:\nGenerate a SQL query that answers the questioq `Total Entry transactions OF Seaport for date 12th July 2023`.\nThis query will run on a database whose schema is represented in this string:\nCREATE TABLE ENEX_MASTER  --table for entry exit \n(\n  EMA_SEQ                  NUMBER(5), -- Unique ID for each SPM_PERSON_NO\n  SPM_PERSON_NO            NUMBER(12), --Unique number per each person\n  NAT_CODE                 NUMBER(3),\n  EBO_CODE                 NUMBER(4),\n  ETT_CODE                 NUMBER(3),\n  SFU_CODE                 NU

In [43]:
question = "Total Entry transactions OF Seaport of Female passengers for date 12th July 2023"

In [47]:
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT sum(enex_master.ema_seq) AS total_entry_transactions
FROM   enex_master join enex_border on enex_master.ebo_code = enex_border.ebo_code
WHERE  enex_master.ema_date = '2023-07-12'
   and enex_border.ebo_type = 2;


In [48]:
print(outputs)

["### Instructions:\nYour task is convert a question into a SQL query, given a Postgres database schema.\nAdhere to these rules:\n- **Deliberately go through the question and database schema word by word** to appropriately answer the question\n- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.\n- When creating a ratio, always cast the numerator as float\n\n### Input:\nGenerate a SQL query that answers the questioq `Total Entry transactions OF Seaport for date 12th July 2023`.\nThis query will run on a database whose schema is represented in this string:\nCREATE TABLE ENEX_MASTER  --table for entry exit \n(\n  EMA_SEQ                  NUMBER(5), -- Unique ID for each SPM_PERSON_NO\n  SPM_PERSON_NO            NUMBER(12), --Unique number per each person\n  NAT_CODE                 NUMBER(3),\n  EBO_CODE                 NUMBER(4),\n  ETT_CODE                 NUMBER(3),\n  SFU_CODE                 NU